In [2]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import uniform
import tqdm

In [3]:
# CI function
def CI(data,alpha):
    sample_mean=np.mean(data) # data is a list!
    sample_sigma=np.std(data)
    critical_point = norm.ppf(1-alpha/2)
    LB=sample_mean-critical_point*sample_sigma/np.sqrt(len(data))
    UB=sample_mean+critical_point*sample_sigma/np.sqrt(len(data))
    return LB,UB

# Question 2 :

In [126]:
'''
Exit time T follow pdf f(x)=x/18 0<x<6
If T<6, Exit payoff: p*IPO payoff+(1-p)*M&A payoff 
Exit value above 1B would be IPO, all other are M&A
If T=6, exit would be M&A
Series B / M&A payoff = formula 1
Series B / IPO payoff = X*Ib/Pb
Also need to check if XT>1000 or not, 
'''
# parameters
Pa,Pb,Ia,Ib=450,1000,50,100
share_new,share_old,share_current=0.1,0.1,0.8
r,sigma=0.025,0.9
size=1000000

X0=600 # ERROR! Can be adjusted!

U=np.random.uniform(0,1,size)
T=np.sqrt(36*U)

Z=np.random.normal(0,1,size)
X=X0*np.exp((r-0.5*(sigma**2))*T+sigma*np.sqrt(T)*Z)

Ib_vector=np.ones(size)*Ib

# 目前是用X去判斷 不知道T如何

Merge = np.max((np.min([Ia*X[np.where(X<1000)]/(Ia+Ib),np.ones(len(np.where(X<1000)[0]))*Ib],axis=0),X[np.where(X<1000)]*Ib/Pb),axis=0)
IPO = X[np.where(X>=1000)]*Ib/Pb
payoff=np.append(Merge,IPO)*np.exp(-r*T)

print(np.mean(payoff),CI(payoff,0.05))



81.03068794835937 (80.31308674346641, 81.74828915325233)


In [ ]:
# parameters
r,div,sigma,rho=0.05,0.02,0.3,0.2
S10,S20,S30,K,T=100,100,100,100,1
N,size = 252,100000 # N:steps size:simulation times
h=T/N
# simulate standard normal
Z1=np.random.normal(0,1,(N,size))
Z2=np.random.normal(0,1,(N,size))
Z3=np.random.normal(0,1,(N,size))
X1=Z1
X2=rho*Z1+np.sqrt(1-(rho**2))*Z2
X3=rho*Z1+((rho-rho**2)/np.sqrt(1-rho**2))*Z2+(np.sqrt(1-rho**2-((rho-rho**2)**2)/(1-rho**2)))*Z3
# simulate stock price: St=S0*exp{(miu-0.5*sigma^2)*t+sigma*Wt}
S1=np.ones((1,size))*S10
S2=np.ones((1,size))*S20
S3=np.ones((1,size))*S30
zero = np.zeros(size)
for i in tqdm.tqdm(range(N)):
    a=S1[i,:]*np.exp((r-div-(sigma**2)*0.5)*(h)+sigma*np.sqrt(h)*X1[i,:])
    b=S2[i,:]*np.exp((r-div-(sigma**2)*0.5)*(h)+sigma*np.sqrt(h)*X2[i,:])
    c=S3[i,:]*np.exp((r-div-(sigma**2)*0.5)*(h)+sigma*np.sqrt(h)*X3[i,:])
    S1=np.append(S1,[a],axis=0)
    S2=np.append(S2,[b],axis=0)
    S3=np.append(S3,[c],axis=0)
payoff=np.max([S1[-1,:]+S2[-1,:]-S3[-1,:]-K,zero],axis=0) 
# Discount to now